# Import

In [1]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 15.3 MB/s eta 0:00:00


In [2]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load dataset & model

In [38]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [35]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/insdn/insdn_mlp_bin.h5', compile=True)

In [39]:
x_normal = df[df['Label'] == 0].reset_index(drop=True)

In [40]:
x_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54797 entries, 0 to 54796
Data columns (total 77 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      54797 non-null  float64
 1   Tot Fwd Pkts       54797 non-null  float64
 2   Tot Bwd Pkts       54797 non-null  float64
 3   TotLen Fwd Pkts    54797 non-null  float64
 4   TotLen Bwd Pkts    54797 non-null  float64
 5   Fwd Pkt Len Max    54797 non-null  float64
 6   Fwd Pkt Len Min    54797 non-null  float64
 7   Fwd Pkt Len Mean   54797 non-null  float64
 8   Fwd Pkt Len Std    54797 non-null  float64
 9   Bwd Pkt Len Max    54797 non-null  float64
 10  Bwd Pkt Len Min    54797 non-null  float64
 11  Bwd Pkt Len Mean   54797 non-null  float64
 12  Bwd Pkt Len Std    54797 non-null  float64
 13  Flow Byts/s        54797 non-null  float64
 14  Flow Pkts/s        54797 non-null  float64
 15  Flow IAT Mean      54797 non-null  float64
 16  Flow IAT Std       547

In [ ]:
x_normal['Label'].value_counts()

In [ ]:
df = df.drop(columns=['Label'])

In [ ]:
x_normal = x_normal.drop(columns=['Label'])

In [49]:
feature_list = x_normal.columns

# SHAP

In [50]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100))

In [51]:
normal_1500 = x_normal.head(1500)

In [52]:
shap_values = explainer.shap_values(normal_1500)

  0%|          | 0/1500 [00:00<?, ?it/s]

In [53]:
data_df = pd.DataFrame(shap_values[0])

In [54]:
data_df.to_csv('/content/drive/MyDrive/1500_samples_[0].csv', index=False)

In [55]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,-0.024007,0.000000,0.010930,0.007230,0.014859,-0.009763,-0.008111,0.011861,0.000000,0.041048,...,-0.007639,0.0,0.008328,-0.026166,0.019688,-0.009793,0.013286,0.014610,0.02187,-0.007478
1,0.000000,-0.000659,0.000000,0.005447,0.000000,-0.009084,-0.004624,-0.012858,-0.002050,-0.005433,...,-0.006068,0.0,0.000000,0.000000,-0.002505,0.009195,0.010743,0.000000,0.00000,0.000000
2,-0.008410,0.003195,0.015692,0.000000,-0.001917,0.002825,-0.015668,-0.015815,0.000000,0.005682,...,-0.002513,0.0,0.004660,0.000000,-0.008027,0.005925,0.002991,0.002691,0.00338,-0.003728
3,0.004181,-0.010780,-0.002135,0.000000,0.004707,-0.012816,0.005066,-0.017118,0.002889,0.000000,...,0.000000,0.0,0.000000,0.000000,0.008357,-0.002734,0.004717,0.000000,0.00000,0.005926
4,-0.004836,0.000000,0.000000,0.006560,-0.005721,-0.023632,0.000000,-0.016735,-0.016155,0.000000,...,-0.007222,0.0,0.007713,-0.005103,0.000000,-0.009165,0.000000,-0.007764,0.00000,0.000000


In [56]:
data_dict = {}
for idx in range(0, 1500):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Normal sample ' + str(idx)] = row_values

In [57]:
print(data_dict)

{'Normal sample 0': ['Bwd IAT Max', 'Bwd Pkt Len Max', 'ACK Flag Cnt', 'Bwd IAT Tot', 'Bwd Header Len', 'Idle Max', 'Active Max', 'Flow IAT Max', 'TotLen Bwd Pkts', 'Idle Std'], 'Normal sample 1': ['Init Bwd Win Byts', 'Idle Mean', 'Active Min', 'Fwd Header Len', 'Pkt Len Var', 'TotLen Fwd Pkts', 'Flow Duration', 'Tot Bwd Pkts', 'TotLen Bwd Pkts', 'Bwd Pkt Len Mean'], 'Normal sample 2': ['Tot Bwd Pkts', 'Init Bwd Win Byts', 'Fwd Header Len', 'Subflow Bwd Pkts', 'Bwd Pkt Len Std', 'Subflow Fwd Pkts', 'Active Min', 'Bwd Pkt Len Max', 'Pkt Len Var', 'Active Mean'], 'Normal sample 3': ['Init Bwd Win Byts', 'Active Max', 'Bwd Seg Size Avg', 'Subflow Bwd Byts', 'Idle Min', 'Bwd IAT Tot', 'Bwd Pkt Len Mean', 'Fwd Pkt Len Min', 'Fwd Header Len', 'Idle Mean'], 'Normal sample 4': ['SYN Flag Cnt', 'ACK Flag Cnt', 'Bwd Pkt Len Std', 'Active Mean', 'Bwd Pkt Len Mean', 'Fwd Pkts/s', 'TotLen Fwd Pkts', 'Flow IAT Min', 'Fwd IAT Tot', 'Bwd URG Flags'], 'Normal sample 5': ['Init Bwd Win Byts', 'Fwd IAT 

In [58]:
len(data_dict)

1500

# New whitelist: 38 top features with over 10k freq each

In [59]:
df = pd.DataFrame.from_dict(data_dict, orient='index', columns=['Feature 1', 'Feature 2', 'Feature 3', 'Feature 4', 'Feature 5', 'Feature 6', 'Feature 7', 'Feature 8', 'Feature 9', 'Feature 10'])

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, Normal sample 0 to Normal sample 1499
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Feature 1   1500 non-null   object
 1   Feature 2   1500 non-null   object
 2   Feature 3   1500 non-null   object
 3   Feature 4   1500 non-null   object
 4   Feature 5   1500 non-null   object
 5   Feature 6   1500 non-null   object
 6   Feature 7   1500 non-null   object
 7   Feature 8   1500 non-null   object
 8   Feature 9   1500 non-null   object
 9   Feature 10  1500 non-null   object
dtypes: object(10)
memory usage: 128.9+ KB


In [61]:
df.head()

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10
Normal sample 0,Bwd IAT Max,Bwd Pkt Len Max,ACK Flag Cnt,Bwd IAT Tot,Bwd Header Len,Idle Max,Active Max,Flow IAT Max,TotLen Bwd Pkts,Idle Std
Normal sample 1,Init Bwd Win Byts,Idle Mean,Active Min,Fwd Header Len,Pkt Len Var,TotLen Fwd Pkts,Flow Duration,Tot Bwd Pkts,TotLen Bwd Pkts,Bwd Pkt Len Mean
Normal sample 2,Tot Bwd Pkts,Init Bwd Win Byts,Fwd Header Len,Subflow Bwd Pkts,Bwd Pkt Len Std,Subflow Fwd Pkts,Active Min,Bwd Pkt Len Max,Pkt Len Var,Active Mean
Normal sample 3,Init Bwd Win Byts,Active Max,Bwd Seg Size Avg,Subflow Bwd Byts,Idle Min,Bwd IAT Tot,Bwd Pkt Len Mean,Fwd Pkt Len Min,Fwd Header Len,Idle Mean
Normal sample 4,SYN Flag Cnt,ACK Flag Cnt,Bwd Pkt Len Std,Active Mean,Bwd Pkt Len Mean,Fwd Pkts/s,TotLen Fwd Pkts,Flow IAT Min,Fwd IAT Tot,Bwd URG Flags


In [62]:
df.to_csv('/content/drive/MyDrive/MLP_top10_of_each_normalfile.csv')

In [63]:
freq = {}
for column in df.columns:
  for feature in df[column]:
    if feature in freq:
      continue
    else:
      count = sum(df[column_2] == feature for column_2 in df.columns)
      freq[feature] = count.sum()
      print(freq[feature])

340
684
337
293
383
462
242
147
316
535
604
76
196
248
571
169
178
103
535
181
178
162
162
32
187
159
173
4
155
206
176
358
145
444
364
182
171
181
200
268
185
253
188
219
141
201
219
181
191
273
329
169
324
144
187
198
172
183
260
143
75
110
88
52
4
4


In [64]:
len(freq)

66

In [65]:
freq_df = pd.DataFrame(list(freq.items()), columns=['Feature', 'Frequency'])
freq_df.head()

,Feature,Frequency
0,Bwd IAT Max,340
1,Init Bwd Win Byts,684
2,Tot Bwd Pkts,337
3,SYN Flag Cnt,293
4,Pkt Len Min,383


In [66]:
freq_df.sort_values(by=['Frequency'], ascending=False, inplace=True)
freq_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 1 to 65
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Feature    66 non-null     object
 1   Frequency  66 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


In [67]:
freq_df.head(24)

,Feature,Frequency
1,Init Bwd Win Byts,684
10,Idle Max,604
14,Bwd Pkt Len Max,571
18,Idle Mean,535
9,Idle Min,535
5,Bwd Pkt Len Mean,462
33,Bwd Pkt Len Std,444
4,Pkt Len Min,383
34,Tot Fwd Pkts,364
31,Bwd Seg Size Avg,358


In [68]:
freq_df[:38].to_csv('/content/drive/MyDrive/MLP_top24_InSDN.csv', index=False)

# Detect normal samples

In [69]:
whitelist = freq_df.iloc[:, 0][:38].tolist()
print(whitelist)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Idle Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Pkt Len Min', 'Tot Fwd Pkts', 'Bwd Seg Size Avg', 'Bwd IAT Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Tot', 'SYN Flag Cnt', 'Flow Pkts/s', 'Bwd IAT Std', 'Bwd Pkt Len Min', 'Fwd Pkt Len Min', 'Flow Duration', 'Down/Up Ratio', 'Flow IAT Min', 'Fwd Pkt Len Std', 'Fwd Pkts/s', 'Active Min', 'Fwd IAT Std', 'Flow IAT Mean', 'Fwd IAT Mean', 'Bwd Header Len', 'Pkt Len Var', 'Active Mean', 'Fwd IAT Tot', 'Active Max', 'Idle Std', 'Active Std', 'Flow IAT Std', 'Subflow Bwd Pkts']


In [70]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:1308


In [71]:
whitelist = freq_df.iloc[:, 0][:24].tolist()
print(whitelist)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Idle Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Pkt Len Min', 'Tot Fwd Pkts', 'Bwd Seg Size Avg', 'Bwd IAT Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Tot', 'SYN Flag Cnt', 'Flow Pkts/s', 'Bwd IAT Std', 'Bwd Pkt Len Min', 'Fwd Pkt Len Min', 'Flow Duration', 'Down/Up Ratio', 'Flow IAT Min', 'Fwd Pkt Len Std']


In [72]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:1482


## False positive of XAI on normal samples of trainset:
top 38: 1308/1500

top 24: 1482/1500